In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from gensim.models import Word2Vec
import datetime
from dateutil.relativedelta import relativedelta

C:\Users\Lassana Diabira\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
df=pd.read_csv("data.csv")

C:\Users\Lassana Diabira\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (2,3,6,7,8,9,12,13,14,15,17,19,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df['job_created_date'] =  pd.to_datetime(df['job_created_date'])
df['job_opening_date'] =  pd.to_datetime(df['job_opening_date'])
df['job_closing_date'] =  pd.to_datetime(df['job_closing_date'])

In [4]:
df=df[df.user_id.notnull()]

In [5]:
def get_all_users_id(df):
    users_id=df.user_id.values
    users_id=list(set(users_id))
    return users_id

In [6]:
all_users=get_all_users_id(df)

In [7]:
len(all_users)

165822

In [8]:
df=df[df.job_id.notnull()]

In [9]:
df.shape

(836373, 31)

In [10]:
df.tail(1)

,Unnamed: 0,user_id,job_created_date,job_application_status,job_id,employer_id,job_opening_date,job_closing_date,job_short_description,job_description,...,course_includes_industrial_placement,graduation_year,right_to_work_uk,degree_subject_id,university_id,university_name,university_type,degree_subject_name,degree_subjectgroup_id,degree_subjectgroup_title
836372,671110,154122.0,2018-08-31 12:13:24,Initiated,2459.0,131.0,2018-08-17,2018-10-21,Greenhill & Co. is a leading independent inves...,<h2>The role</h2>\r\r\n\r\r\n<p>Genuine analys...,...,False,2019.0,True,751.0,345.0,University of Cambridge,2.0,Geography,4.0,Geography


In [11]:
df.head(1)

,Unnamed: 0,user_id,job_created_date,job_application_status,job_id,employer_id,job_opening_date,job_closing_date,job_short_description,job_description,...,course_includes_industrial_placement,graduation_year,right_to_work_uk,degree_subject_id,university_id,university_name,university_type,degree_subject_name,degree_subjectgroup_id,degree_subjectgroup_title
0,0,75898.0,2017-09-01 01:00:02,Initiated,214.0,25.0,2017-03-27,2017-08-31,Think of our Summer Vacation Scheme in 2018 as...,<p>Apply now to fast track your application to...,...,False,2019.0,True,1371.0,381.0,University of Manchester,2.0,Spanish and Business,26.0,Modern Languages


In [12]:
def get_users_id(df):
    users_id=df.user_id.values
    users_id=list(set(users_id))
    return users_id

In [13]:
users_id=get_users_id(df)

In [14]:
len(users_id)

20812

In [15]:
def get_jobs_id(df):
    jobs_id=df.job_id.values
    jobs_id=list(set(jobs_id))
    return jobs_id

In [16]:
jobs_id=get_jobs_id(df)

In [17]:
len(jobs_id)

2988

In [18]:
def get_jobs_per_user(users_id, df):
    n=len(users_id)
    print("n :",n)
    print("users collected")
    jobs_per_user={}
    for i,user in enumerate(users_id):
        if i%1000==0:
            print("progress :", i/n)
        x=df[df["user_id"]==user]
        x=x[x.job_id.notnull()]
        if x.shape[0]>0:
            job_applications=x["job_id"].values
            job_applications=list(set(job_applications))
            jobs_per_user[str(int(user))]=job_applications
    print("Jobs per user collected")
    return jobs_per_user

In [19]:
jobs_per_user=get_jobs_per_user(users_id, df)

n : 20812
users collected
progress : 0.0
progress : 0.04804920238324044
progress : 0.09609840476648088
progress : 0.1441476071497213
progress : 0.19219680953296175
progress : 0.24024601191620218
progress : 0.2882952142994426
progress : 0.33634441668268306
progress : 0.3843936190659235
progress : 0.43244282144916396
progress : 0.48049202383240436
progress : 0.5285412262156448
progress : 0.5765904285988852
progress : 0.6246396309821257
progress : 0.6726888333653661
progress : 0.7207380357486066
progress : 0.768787238131847
progress : 0.8168364405150874
progress : 0.8648856428983279
progress : 0.9129348452815683
progress : 0.9609840476648087
Jobs per user collected


In [20]:
def get_jobs_to_recommend(all_users,df,jobs_per_user):
    n=len(all_users)
    recommendations_per_user={}
    now = datetime.datetime.now()
    #d=df[df['job_created_date']>=now-relativedelta(years=1)]
    c=df[df['job_closing_date'].isnull()]
    d=df[df['job_closing_date']>=now]
    d=d[d.job_id.notnull()]
    jobs_to_apply_to=list(d["job_id"].values)+list(c["job_id"].values)
    jobs_to_apply_to=list(set(jobs_to_apply_to))
    print("number of jobs to apply to :",len(jobs_to_apply_to))
    print("recommendations started")
    for i,user in enumerate(all_users):
        if i%10000==0:
            print("progress :", i/n)
        try:
            job_applications=jobs_per_user[str(int(user))]
            recommendations=list(set(jobs_to_apply_to) - set(job_applications))
            recommendations_per_user[str(int(user))]=recommendations
        except:
            recommendations_per_user[str(int(user))]=jobs_to_apply_to
    return recommendations_per_user

In [21]:
recommendations_per_user=get_jobs_to_recommend(all_users, df, jobs_per_user)

number of jobs to apply to : 294
recommendations started
progress : 0.0
progress : 0.060305628927404084
progress : 0.12061125785480817
progress : 0.18091688678221224
progress : 0.24122251570961634
progress : 0.30152814463702043
progress : 0.3618337735644245
progress : 0.4221394024918286
progress : 0.4824450314192327
progress : 0.5427506603466368
progress : 0.6030562892740409
progress : 0.663361918201445
progress : 0.723667547128849
progress : 0.783973176056253
progress : 0.8442788049836571
progress : 0.9045844339110612
progress : 0.9648900628384653


# create train and test dataset

In [22]:
a=list(jobs_per_user.keys())
train_user_id=[int(i) for i in a]
train_user_applications=list(jobs_per_user.values())

In [23]:
a=list(recommendations_per_user.keys())
test_user_id=[int(i) for i in a]
test_user_applications=list(recommendations_per_user.values())

In [24]:
class users:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.user_id=self.get_ids(d)
        self.length_of_course=self.get_length_of_course(d)
        self.gender=self.get_gender(d)
        self.ethnicity=self.get_ethnicity(d)
        self.school_type=self.get_school_type(d)
        self.course_includes_industrial_placement=self.get_course_includes_industrial_placement(d)
        self.graduation_year=self.get_graduation_year(d)
        self.right_to_work_uk=self.get_right_to_work_uk(d)
        self.degree_subject_id=self.get_degree_subject_id(d)
        self.university_id=self.get_university_id(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["user_id"]))
        return res
    
    def get_length_of_course(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["length_of_course"]))
            except TypeError:
                res.append(None)
        return res
    
    def get_gender(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["gender"])
        return res
    
    def get_ethnicity(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["ethnicity"])
        return res
    
    def get_school_type(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["school_type"])
        return res
    
    def get_course_includes_industrial_placement(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["course_includes_industrial_placement"])
        return res
    
    def get_graduation_year(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["graduation_year"]))
            except TypeError:
                res.append(None)
        return res
    
    def get_right_to_work_uk(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["right_to_work_uk"])
        return res
    
    def get_degree_subject_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["degree_subject_id"]))
            except TypeError:
                res.append(None)
        return res
    
    def get_university_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["university_id"]))
            except TypeError:
                res.append(None)
        return res
    
    def create_df(self):
        dd={}
        dd["user_id"]=self.user_id
        dd["length_of_course"]=self.length_of_course
        dd["gender"]=self.gender
        dd["ethnicity"]=self.ethnicity
        dd["school_type"]=self.school_type
        dd["course_includes_industrial_placement"]=self.course_includes_industrial_placement
        dd["graduation_year"]=self.graduation_year
        dd["right_to_work_uk"]=self.right_to_work_uk
        dd["degree_subject_id"]=self.degree_subject_id
        dd["university_id"]=self.university_id
        self.df=pd.DataFrame(dd)

In [25]:
user=users("users_profile.json", "users_profile")
user_df=user.df

In [26]:
class jobs:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.job_id=self.get_ids(d)
        self.opening_date=self.get_opening_date(d)
        self.closing_date=self.get_closing_date(d)
        self.short_description=self.get_short_description(d)
        self.description=self.get_description(d)
        self.job_type=self.get_job_type(d)
        self.employer_id=self.get_employer_id(d)
        self.search_rank=self.get_search_rank(d)
        self.title=self.get_title(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["id"]))
        return res
    
    def get_opening_date(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["opening_date"])
        return res
    
    def get_closing_date(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["closing_date"])
        return res
    
    def get_short_description(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["description_short"])
        return res
    
    def get_description(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["description"])
        return res
    
    def get_job_type(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["job_type"]))
        return res
    
    def get_employer_id(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["employer_id"]))
        return res
    
    def get_search_rank(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["search_rank"]))
        return res
    
    def get_title(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["title"])
        return res
    
    def create_df(self):
        dd={}
        dd["job_id"]=self.job_id
        dd["job_opening_date"]=self.opening_date
        dd["job_closing_date"]=self.closing_date
        dd["job_short_description"]=self.short_description
        dd["job_description"]=self.description
        dd["job_type"]=self.job_type
        dd["employer_id"]=self.employer_id
        dd["search_rank"]=self.search_rank
        dd["job_title"]=self.title
        self.df=pd.DataFrame(dd)

In [27]:
job=jobs("job.json", "job")
job_df=job.df

In [28]:
class employers_job_sectors:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.employer_id=self.get_employer_id(d)
        self.jobsector_id=self.get_jobsector_id(d)
        self.create_df()
    
    def get_employer_id(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["employer_id"]))
        return res
    
    def get_jobsector_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["jobsector_id"]))
            except TypeError:
                res.append(None)
        return res
    
    def create_df(self):
        dd={}
        dd["employer_id"]=self.employer_id
        dd["employer_sector_id"]=self.jobsector_id
        self.df=pd.DataFrame(dd)

In [29]:
employer_job_sector=employers_job_sectors("employers_sectors.json", "employers_sectors")
employer_job_sector_df=employer_job_sector.df

In [30]:
class jobsectors:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.title=self.get_title(d)
        self.jobsector_id=self.get_jobsector_id(d)
        self.create_df()
    
    def get_title(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["title"])
        return res
    
    def get_jobsector_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["id"]))
            except TypeError:
                res.append(None)
        return res
    
    def create_df(self):
        dd={}
        dd["jobsector_title"]=self.title
        dd["jobsector_id"]=self.jobsector_id
        self.df=pd.DataFrame(dd)

In [31]:
jobsector=jobsectors("jobsectors.json", "job_sectors")
jobsector_df=jobsector.df

In [32]:
class job_sectors:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.job_id=self.get_job_id(d)
        self.jobsector_id=self.get_jobsector_id(d)
        self.create_df()
    
    def get_job_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["job_id"]))
            except TypeError:
                res.append(None)
        return res
    
    def get_jobsector_id(self, x):
        res=[]
        for i in range(len(x)):
            try:
                res.append(int(x[i]["jobsector_id"]))
            except TypeError:
                res.append(None)
        return res
    
    def create_df(self):
        dd={}
        dd["job_id"]=self.job_id
        dd["jobsector_id"]=self.jobsector_id
        self.df=pd.DataFrame(dd)

In [33]:
job_sector=job_sectors("job_sectors.json", "job_sectors")
job_sector_df=job_sector.df

In [34]:
class employers:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.employer_id=self.get_ids(d)
        self.title=self.get_title(d)
        self.description_short=self.get_description_short(d)
        self.description=self.get_description(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["id"]))
        return res
    
    def get_title(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["title"])
        return res
    
    def get_description_short(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["description_short"])
        return res
    
    
    def get_description(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["description"])
        return res
    
    
    def create_df(self):
        dd={}
        dd["employer_id"]=self.employer_id
        dd["employer_title"]=self.title
        dd["employer_description_short"]=self.description_short
        dd["employer_description"]=self.description
        self.df=pd.DataFrame(dd)

In [35]:
employer=employers("employer.json", "employer")
employer_df=employer.df

In [36]:
class university:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.university_id=self.get_ids(d)
        self.university_name=self.get_university_name(d)
        self.university_type=self.get_university_type(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["id"]))
        return res
    
    def get_university_name(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["name"])
        return res
    
    def get_university_type(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["university_type"]))
        return res
    
    def create_df(self):
        dd={}
        dd["university_id"]=self.university_id
        dd["university_name"]=self.university_name
        dd["university_type"]=self.university_type
        self.df=pd.DataFrame(dd)

In [37]:
uni=university("university.json", "university")
uni_df=uni.df

In [38]:
class subject:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.subject_id=self.get_ids(d)
        self.subject_name=self.get_subject_name(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["id"]))
        return res
    
    def get_subject_name(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["name"])
        return res
    
    
    def create_df(self):
        dd={}
        dd["degree_subject_id"]=self.subject_id
        dd["degree_subject_name"]=self.subject_name
        self.df=pd.DataFrame(dd)

In [39]:
sub=subject("subject.json", "subject")
subject_df=sub.df

In [40]:
class subject_groups:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.subject_id=self.get_ids(d)
        self.subjectgroup_id=self.get_subjectgroup_id(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["subject_id"]))
        return res
    
    def get_subjectgroup_id(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["subjectgroup_id"]))
        return res
    
    
    def create_df(self):
        dd={}
        dd["degree_subject_id"]=self.subject_id
        dd["degree_subjectgroup_id"]=self.subjectgroup_id
        self.df=pd.DataFrame(dd)

In [41]:
subgroup=subject_groups("subject_groups.json", "subject_groups")
subjectgroup_df=subgroup.df

In [42]:
class subjectgroup:
    
    def __init__(self, filename, filtre):
        data=pd.read_json(filename)
        d=data[filtre].values
        self.subjectgroup_id=self.get_ids(d)
        self.subjectgroup_title=self.get_subject_name(d)
        self.create_df()
    
    def get_ids(self, x):
        res=[]
        for i in range(len(x)):
            res.append(int(x[i]["id"]))
        return res
    
    def get_subject_name(self, x):
        res=[]
        for i in range(len(x)):
            res.append(x[i]["title"])
        return res
    
    
    def create_df(self):
        dd={}
        dd["degree_subjectgroup_id"]=self.subjectgroup_id
        dd["degree_subjectgroup_title"]=self.subjectgroup_title
        self.df=pd.DataFrame(dd)

In [43]:
subgrouptitle=subjectgroup("subjectgroup.json", "subjectgroup")
subjectgrouptitle_df=subgrouptitle.df

In [44]:
job_df=pd.merge(job_df, employer_df, on='employer_id', how='outer')

In [45]:
job_df=pd.merge(job_df, employer_job_sector_df, on='employer_id', how='outer')

In [46]:
job_df=pd.merge(job_df, jobsector_df, left_on = 'employer_sector_id', right_on = 'jobsector_id', how='outer')

In [47]:
job_df=job_df.rename(index=str, columns={"jobsector_title": "employer_sector_title"})

In [48]:
job_df.drop("jobsector_id", axis=1, inplace=True)

In [49]:
job_df=pd.merge(job_df, job_sector_df, on='job_id', how='outer')

In [50]:
job_df=pd.merge(job_df, jobsector_df, on = 'jobsector_id', how='outer')

In [51]:
job_df=job_df.rename(index=str, columns={"jobsector_title": "job_sector_title"})

In [52]:
user_df=pd.merge(user_df, uni_df, on='university_id', how='outer')

In [53]:
user_df=pd.merge(user_df, subject_df, on='degree_subject_id', how='outer')

In [54]:
user_df=pd.merge(user_df, subjectgroup_df, on='degree_subject_id', how='outer')

In [55]:
#user_df=pd.merge(user_df, subjectgrouptitle_df, on='degree_subjectgroup_id', how='outer')

In [56]:
user_df.drop("degree_subjectgroup_id", axis=1, inplace=True)

In [57]:
user_df=user_df.drop_duplicates(subset=user_df.columns)

In [58]:
user_df[user_df["user_id"]==98304]

,user_id,length_of_course,gender,ethnicity,school_type,course_includes_industrial_placement,graduation_year,right_to_work_uk,degree_subject_id,university_id,university_name,university_type,degree_subject_name
94492,98304.0,NaN,Female,None,None,false,2019.0,true,835.0,196.0,Durham University,2.0,History and Politics


In [59]:
#job_df=job_df[job_df["employer_sector_title"]!=job_df["job_sector_title"]]

In [60]:
job_df[job_df["job_id"]==3163.0]["job_opening_date"]

7871    2017-10-01
8827    2017-10-01
Name: job_opening_date, dtype: object

In [70]:
job_df.columns

Index(['job_id', 'job_opening_date', 'job_closing_date',
       'job_short_description', 'job_description', 'job_type', 'employer_id',
       'search_rank', 'job_title', 'employer_title',
       'employer_description_short', 'employer_description',
       'employer_sector_id', 'employer_sector_title', 'jobsector_id',
       'job_sector_title'],
      dtype='object')

In [61]:
user_df.shape

(165871, 13)

In [62]:
job_df.shape

(14348, 16)

In [79]:
def make_dataset(job_df, user_df, ids, applications):
    n=len(ids)
    d={}
    user_id=[]
    job_id=[]
    employer_id=[]
    job_opening_date=[]
    job_closing_date=[]
    job_description=[]
    job_type=[]
    job_title=[]
    employer_title=[]
    employer_description=[]
    employer_sector_id=[]
    employer_sector_title=[]
    jobsector_id=[]
    job_sector_title=[]
    gender=[]
    course_includes_industrial_placement=[]
    graduation_year=[]
    right_to_work_uk=[]
    degree_subject_id=[]
    university_id=[]
    university_name=[]
    university_type=[]
    degree_subject_name=[]
    for i in range(n):
        if i%1000==0:
            print("progress :",i/n)
        user_applications=applications[i]
        nb_applications=len(user_applications)
        for j in range(nb_applications):
            user_id.append(ids[i])
            job_id.append(user_applications[j])
            user_info=user_df[user_df["user_id"]==ids[i]]
            job_infos=job_df[job_df["job_id"]==user_applications[j]]
            employer_id.append(job_infos["employer_id"].iloc[0])
            job_opening_date.append(job_infos["job_opening_date"].iloc[0])
            job_closing_date.append(job_infos["job_closing_date"].iloc[0])
            job_description.append(job_infos["job_description"].iloc[0])
            job_type.append(job_infos["job_type"].iloc[0])
            job_title.append(job_infos["job_title"].iloc[0])
            employer_title.append(job_infos["employer_title"].iloc[0])
            employer_description.append(job_infos["employer_description"].iloc[0])
            employer_sector_id.append(job_infos["employer_sector_id"].iloc[0])
            employer_sector_title.append(job_infos["employer_sector_title"].iloc[0])
            jobsector_id.append(job_infos["jobsector_id"].iloc[0])
            job_sector_title.append(job_infos["job_sector_title"].iloc[0])
            try:
                gender.append(user_info["gender"].iloc[0])
            except:
                gender.append(None)
            try:
                course_includes_industrial_placement.append(user_info["course_includes_industrial_placement"].iloc[0])
            except:
                course_includes_industrial_placement.append(None)
            try:
                graduation_year.append(user_info["graduation_year"].iloc[0])
            except:
                graduation_year.append(None)
            try:
                right_to_work_uk.append(user_info["right_to_work_uk"].iloc[0])
            except:
                right_to_work_uk.append(None)
            try:
                degree_subject_id.append(user_info["degree_subject_id"].iloc[0])
            except:
                degree_subject_id.append(None)
            try:
                university_id.append(user_info["university_id"].iloc[0])
            except:
                university_id.append(None)
            try:
                university_name.append(user_info["university_name"].iloc[0])
            except:
                university_name.append(None)
            try:
                university_type.append(user_info["university_type"].iloc[0])
            except:
                university_type.append(None)
            try:
                degree_subject_name.append(user_info["degree_subject_name"].iloc[0])
            except:
                degree_subject_name.append(None)
    d["user_id"]=user_id
    d["job_id"]=job_id
    d["employer_id"]=employer_id
    d["job_opening_date"]=job_opening_date
    d["job_closing_date"]=job_closing_date
    d["job_description"]=job_description
    d["job_type"]=job_type
    d["job_title"]=job_title
    d["employer_title"]=employer_title
    d["employer_description"]=employer_description
    d["employer_sector_id"]=employer_sector_id
    d["employer_sector_title"]=employer_sector_title
    d["jobsector_id"]=jobsector_id
    d["job_sector_title"]=job_sector_title
    d["gender"]=gender
    d["course_includes_industrial_placement"]=course_includes_industrial_placement
    d["graduation_year"]=graduation_year
    d["right_to_work_uk"]=right_to_work_uk
    d["degree_subject_id"]=degree_subject_id
    d["university_id"]=university_id
    d["university_name"]=university_name
    d["university_type"]=university_type
    d["degree_subject_name"]=degree_subject_name
    df=pd.DataFrame(d)
    return df

In [80]:
training_set=make_dataset(job_df, user_df, train_user_id, train_user_applications)

progress : 0.0
progress : 0.04804920238324044
progress : 0.09609840476648088
progress : 0.1441476071497213
progress : 0.19219680953296175
progress : 0.24024601191620218
progress : 0.2882952142994426
progress : 0.33634441668268306
progress : 0.3843936190659235
progress : 0.43244282144916396
progress : 0.48049202383240436
progress : 0.5285412262156448
progress : 0.5765904285988852
progress : 0.6246396309821257
progress : 0.6726888333653661
progress : 0.7207380357486066
progress : 0.768787238131847
progress : 0.8168364405150874
progress : 0.8648856428983279
progress : 0.9129348452815683
progress : 0.9609840476648087


In [81]:
training_set.shape

(125136, 23)

In [84]:
training_set.columns

Index(['user_id', 'job_id', 'employer_id', 'job_opening_date',
       'job_closing_date', 'job_description', 'job_type', 'job_title',
       'employer_title', 'employer_description', 'employer_sector_id',
       'employer_sector_title', 'jobsector_id', 'job_sector_title', 'gender',
       'course_includes_industrial_placement', 'graduation_year',
       'right_to_work_uk', 'degree_subject_id', 'university_id',
       'university_name', 'university_type', 'degree_subject_name'],
      dtype='object')

In [83]:
test=training_set[training_set]

,user_id,job_id,employer_id,job_opening_date,job_closing_date,job_description,job_type,job_title,employer_title,employer_description,...,job_sector_title,gender,course_includes_industrial_placement,graduation_year,right_to_work_uk,degree_subject_id,university_id,university_name,university_type,degree_subject_name
0,98304,3163.0,208.0,2017-10-01,2018-11-19,"<h2 style=""margin-left: 0px !important;"">Overv...",1.0,UK Sales Graduate Job 2019,P&G,"<p>Nearly five billion times a day, P&amp;G br...",...,Sales & Commercial,Female,false,2019.0,true,835.0,196.0,Durham University,2.0,History and Politics
1,98304,4533.0,208.0,2018-08-22,2019-01-31,<h2>Overview</h2>\r\n\r\n<p>Want to work with ...,3.0,P&G Ireland Marketing Summer Internship 2019,P&G,"<p>Nearly five billion times a day, P&amp;G br...",...,"Consumer, FMCG & Retail",Female,false,2019.0,true,835.0,196.0,Durham University,2.0,History and Politics
2,98304,2539.0,208.0,2018-08-08,2019-01-31,<div>\r\n<div>\r\n<div>\r\n<p>Want to work wit...,3.0,P&G UK Sales Summer Internship,P&G,"<p>Nearly five billion times a day, P&amp;G br...",...,Sales & Commercial,Female,false,2019.0,true,835.0,196.0,Durham University,2.0,History and Politics
3,65537,2417.0,443.0,None,2018-09-30,<h2>Structure:</h2>\r\n\r\n<p>Our one-week pla...,2.0,Winter Vacation Scheme,Kennedys,<p>Kennedys is a global law firm with expertis...,...,Commercial Law,Male,false,2018.0,true,825.0,422.0,University of Warwick,2.0,History
4,65540,2769.0,451.0,2017-09-08,2017-11-16,<h2>About Man Group</h2>\r\n\r\n<p>Man Group i...,1.0,Investment Management Analyst Programme,Man Group,<p>Man Group is a global active investment man...,...,Financial Services,Male,false,2018.0,true,462.0,327.0,University College London (UCL),2.0,Economics
